# IMPORT MODULES

In [ ]:
import sys

sys.path.append('..')

In [ ]:
from modules.model import DLMA, SBC
from modules.dataset import DataGenerator
import numpy as np
import glob

In [ ]:
from keras.optimizers import Adam

# IMPORT MODEL

In [ ]:
model = SBC(time_steps=3)
optm  = Adam(lr=0.1)
model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])

# IMPORT DATASET

In [ ]:
train_path = '../data/arrays/train/'
dev_path   = '../data/arrays/dev/'
test_path  = '../data/arrays/test/'

partition  = dict()

In [ ]:
train_db = glob.glob(train_path + '*.npy')
dev_db   = glob.glob(dev_path + '*.npy')
test_db  = glob.glob(test_path + '*.npy')

In [ ]:
# Take only the file name, not the full path
partition['train'] = [train_db[i].split('/')[-1] for i in range(len(train_db))]
partition['dev']   = [dev_db[i].split('/')[-1] for i in range(len(dev_db))]
partition['test']  = [test_db[i].split('/')[-1] for i in range(len(test_db))]

In [ ]:
train_generator = DataGenerator(
    partition['train'],
    db_part='train',
    dim=(3, 128),
    batch_size=32,
    n_classes=2,
    shuffle=True
)

dev_generator   = DataGenerator(
    partition['dev'],
    db_part='dev',
    dim=(3, 128),
    batch_size=32,
    n_classes=2,
    shuffle=True
)

test_generator  = DataGenerator(
    partition['test'],
    db_part='test',
    dim=(3, 128),
    batch_size=32,
    n_classes=2,
    shuffle=True
)

# TRAIN

In [ ]:
model.fit_generator(
    generator=train_generator,
    validation_data=dev_generator,
    workers=8,
    epochs=30
)

In [ ]:
model.save_weights('weights_100_epochs.h5')

# TEST MODEL

In [ ]:
model.load_weights('weights_100_epochs.h5')

In [ ]:
model.evaluate_generator(generator=test_generator, verbose=1)

In [ ]:
pred = model.predict_generator(test_generator, verbose=1)
pred = np.argmax(pred, axis=1)